In [27]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

# Задача 1

In [121]:
df = pd.read_csv('names.csv')

In [208]:
X, y = df['name'], df['sex']
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.67)
train_data = tuple((x, y) for x, y in zip(X_train, y_train))
test_data = [(x, y) for x, y in zip(X_test, y_test)]
n = df.size

In [209]:
from collections import defaultdict
from math import log

def train(samples):
    classes, freq = defaultdict(lambda:0), defaultdict(lambda:0)
    for feats, label in samples:
        classes[label] += 1                 # count classes frequencies
        for feat in feats:
            freq[label, feat] += 1          # count features frequencies

    for label, feat in freq:                # normalize features frequencies
        freq[label, feat] /= classes[label]
    for c in classes:                       # normalize classes frequencies
        classes[c] /= len(samples)
#     print(classes, freq)
    return classes, freq                    # return P(C) and P(O|C)

def classify(classifier, feats):
    classes, prob = classifier
    return min(classes.keys(),              # calculate argmin(-log(P(C|O))) -> argmax(P(C|O))
        key = lambda cl: -log(classes[cl]) + sum(-log(prob.get((cl,feat))) for feat in feats))

def get_features(sample): return (sample[-1]) # get last letter

features = [(get_features(feat), label) for feat, label in train_data]
classifier = train(features)

In [210]:
genders = np.array([[name, true_gender, classify(classifier, get_features(name))] for name, true_gender in test_data])

TypeError: must be real number, not NoneType

Судя по всему, в тестовой выборке встречается имя, которое заканчивается на букву, которой не было в тренировочной выборке, а исходный алгоритм не умеет это обрабатывать. Перепишем его так, чтобы это не вызывало ошибки: если буква не встречалась в тренировочных данных, логарифм будет зануляться.

In [212]:
def classify2(classifier, feats):
    classes, prob = classifier
    return min(classes.keys(),              
        key = lambda cl: -log(classes[cl]) + sum(-log(prob.get((cl,feat), 1/n)) for feat in feats))

#в genders лежат имя, настоящий пол и пол, предсказаный с помощью нашего классификатора
genders = np.array([[name, true_gender, classify2(classifier, get_features(name))] for name, true_gender in test_data])
y_pred = genders[:, 2]
from sklearn.metrics import accuracy_score
print('Доля правильных классификаций (accuracy):', accuracy_score(y_test, y_pred))

Доля правильных классификаций (accuracy): 0.7791754756871035


Чтобы оценить, насколько хорошо классификатор справился с задачей, мы можем также использовать метрики precision - отношение числа верно определенных мальчиков (девочек) к общему числу имен, определенных как мальчики (девочки) (в общем случае число верно определенных positive ко всем, определенным как positive), которая показывает точность модели при определении класса boy (girl), и recall - отношение числа верно определенных мальчиков (девочек) к общему числу имен мальчиков (девочек) (в общем случае число верно определенных positive ко всем positive), показывающая способность модели обнаруживать выборки, относящиеся к классу boy (girl). 

In [213]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score

In [214]:
precision_boy = precision_score(y_test, y_pred, pos_label='boy')
recall_boy = recall_score(y_test, y_pred, pos_label='boy')
precision_girl = precision_score(y_test, y_pred, pos_label='girl')
recall_girl = recall_score(y_test, y_pred, pos_label='girl')
print('positive \t boy \t girl')
print(f'precision\t {precision_boy:.3f} \t {precision_girl:.3f}')
print(f'recall\t\t {recall_boy:.3f} \t {recall_girl:.3f}')

positive 	 boy 	 girl
precision	 0.750 	 0.817
recall		 0.840 	 0.718


Теперь построим другую модель - тоже наивный байесовский классификатор, но определение пола происходит на основе первых двух букв имени.

In [215]:
def get_features2(sample): return sample[0] + sample[-1]
get_features2('sample')

'se'

In [216]:
features2 = [(get_features2(feat), label) for feat, label in train_data]
classifier2 = train(features2)
genders2 = np.array([[name, true_gender, classify2(classifier2, get_features2(name))] for name, true_gender in test_data])
y_pred2 = genders2[:, 2]
print('Доля правильных классификаций (accuracy):', accuracy_score(y_test, y_pred2))
precision_boy2 = precision_score(y_test, y_pred2, pos_label='boy')
recall_boy2 = recall_score(y_test, y_pred2, pos_label='boy')
precision_girl2 = precision_score(y_test, y_pred2, pos_label='girl')
recall_girl2 = recall_score(y_test, y_pred2, pos_label='girl')
print('positive \t boy \t girl')
print(f'precision\t {precision_boy2:.3f} \t {precision_girl2:.3f}')
print(f'recall\t\t {recall_boy2:.3f} \t {recall_girl2:.3f}')

Доля правильных классификаций (accuracy): 0.7825581395348837
positive 	 boy 	 girl
precision	 0.766 	 0.801
recall		 0.815 	 0.750


Как мы видим, при добавлении первой буквы имени точность алгоритма классификации не сильно улучшилась, что говорит нам о незначительном влиянии первой буквы на принадлежность имени к определенному полу.

Модифицируем classify: вместо логарифмов будем брать исходные вероятности, вместо минимума максимум, вместо суммы - произведение. По свойствам логарифма, максимум новой функции будет в той же точке, что и минимум старой.

In [217]:
def classify3(classifier, feats):
    classes, prob = classifier
    return max(classes.keys(),              
        key = lambda cl: classes[cl] * np.prod([prob.get((cl,feat), 1/n) for feat in feats]))

In [218]:
genders3 = np.array([[name, true_gender, classify3(classifier2, get_features2(name))] for name, true_gender in test_data])
y_pred3 = genders3[:, 2]
print('Доля правильных классификаций (accuracy):', accuracy_score(y_test, y_pred3))
precision_boy3 = precision_score(y_test, y_pred3, pos_label='boy')
recall_boy3 = recall_score(y_test, y_pred3, pos_label='boy')
precision_girl3 = precision_score(y_test, y_pred3, pos_label='girl')
recall_girl3 = recall_score(y_test, y_pred3, pos_label='girl')
print('positive \t boy \t girl')
print(f'precision\t {precision_boy3:.3f} \t {precision_girl3:.3f}')
print(f'recall\t\t {recall_boy3:.3f} \t {recall_girl3:.3f}')

Доля правильных классификаций (accuracy): 0.7825581395348837
positive 	 boy 	 girl
precision	 0.766 	 0.801
recall		 0.815 	 0.750


Видим, что результаты не изменились, как и ожидалось. Итого, можно сделать вывод, что изменение целевых признаков и классифицирующей функции (кроме тождественного, как в примере выше) влияют на долю правильных классификаций алгоритма.

Применим Гауссовский классификатор (основываясь на последней букве имени). Для этого нужно сначала закодировать буквы в числа с помощью preprocessing.LabelEncoder().

In [219]:
from sklearn.naive_bayes import GaussianNB
from sklearn import preprocessing

le = preprocessing.LabelEncoder()
X_train_coded = le.fit_transform([get_features(x) for x in X_train]).reshape(-1, 1)
X_test_coded = le.fit_transform([get_features(x) for x in X_test]).reshape(-1, 1)

GNB = GaussianNB()
GNB.fit(X_train_coded, y_train)
y_pred_gnb = GNB.predict(X_test_coded)

print('Доля правильных классификаций (accuracy):', accuracy_score(y_test, y_pred_gnb))
precision_boy_gnb = precision_score(y_test, y_pred_gnb, pos_label='boy')
recall_boy_gnb = recall_score(y_test, y_pred_gnb, pos_label='boy')
precision_girl_gnb = precision_score(y_test, y_pred_gnb, pos_label='girl')
recall_girl_gnb = recall_score(y_test, y_pred_gnb, pos_label='girl')
print('positive \t boy \t girl')
print(f'precision\t {precision_boy_gnb:.3f} \t {precision_girl_gnb:.3f}')
print(f'recall\t\t {recall_boy_gnb:.3f} \t {recall_girl_gnb:.3f}')

Доля правильных классификаций (accuracy): 0.733697439511393
positive 	 boy 	 girl
precision	 0.743 	 0.725
recall		 0.717 	 0.750


Как мы видим, результаты гауссовского классификатора оказались неплохими, но чуть хуже, чем наивного.

In [220]:
from sklearn.naive_bayes import MultinomialNB

MNB = MultinomialNB()
MNB.fit(X_train_coded, y_train)
y_pred_mnb = MNB.predict(X_test_coded)

print('Доля правильных классификаций (accuracy):', accuracy_score(y_test, y_pred_mnb))
precision_boy_mnb = precision_score(y_test, y_pred_mnb, pos_label='boy')
recall_boy_mnb = recall_score(y_test, y_pred_mnb, pos_label='boy')
precision_girl_mnb = precision_score(y_test, y_pred_mnb, pos_label='girl')
recall_girl_mnb = recall_score(y_test, y_pred_mnb, pos_label='girl')
print('positive \t boy \t girl')
print(f'precision\t {precision_boy_mnb:.3f} \t {precision_girl_mnb:.3f}')
print(f'recall\t\t {recall_boy_mnb:.3f} \t {recall_girl_mnb:.3f}')

Доля правильных классификаций (accuracy): 0.4986845196147522


C:\Users\Acer\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


positive 	 boy 	 girl
precision	 0.000 	 0.499
recall		 0.000 	 1.000


Видим, что Мультиномиальный классификатор в нашем случае вообще не работает. 

Таким образом, в конкретной задаче для условия классификации по последней букве наивный байесовский классификатор оказался точнее остальных рассматриваемых.

# Задача 2

In [221]:
from sklearn import datasets
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

In [222]:
iris = datasets.load_iris()
X_iris_train, X_iris_test, y_iris_train, y_iris_test = train_test_split(iris.data, iris.target, test_size=0.3, random_state=2)

In [223]:
LDA = LinearDiscriminantAnalysis()
LDA.fit(X_iris_train, y_iris_train)
y_iris_pred = LDA.predict(X_iris_test)

print('accuracy:', accuracy_score(y_iris_test, y_iris_pred))
precision_iris = precision_score(y_iris_test, y_iris_pred, average=None)
recall_iris = recall_score(y_iris_test, y_iris_pred, average=None)
print(f'precision\t {precision_iris}')
print(f'recall\t\t {recall_iris}')

accuracy: 1.0
precision	 [1. 1. 1.]
recall		 [1. 1. 1.]


Видим, что встроенный LDA идеально отрабатывает на этом датасете

Реализуем свой LDA:

In [224]:
def LDA_dimensionality(X, y, k):
    '''
    X - набор данных, y - метка, k - целевой размер
    '''
    label_ = list(set(y))

    X_classify = {}

    for label in label_:
        X1 = np.array([X[i] for i in range(len(X)) if y[i] == label])
        X_classify[label] = X1

    mju = np.mean(X, axis=0)
    mju_classify = {}

    for label in label_:
        mju1 = np.mean(X_classify[label], axis=0)
        mju_classify[label] = mju1

    #St = np.dot((X - mju).T, X - mju)

    Sw = np.zeros((len(mju), len(mju)))  # Вычислить матрицу внутриклассовой дивергенции
    for i in label_:
        Sw += np.dot((X_classify[i] - mju_classify[i]).T,
                     X_classify[i] - mju_classify[i])

    # Sb=St-Sw

    Sb = np.zeros((len(mju), len(mju)))  # Вычислить матрицу внутриклассовой дивергенции
    for i in label_:
        Sb += len(X_classify[i]) * np.dot((mju_classify[i] - mju).reshape(
            (len(mju), 1)), (mju_classify[i] - mju).reshape((1, len(mju))))

    eig_vals, eig_vecs = np.linalg.eig(
        np.linalg.inv(Sw).dot(Sb))  # Вычислить собственное значение и собственную матрицу Sw-1 * Sb

    sorted_indices = np.argsort(eig_vals)
    topk_eig_vecs = eig_vecs[:, sorted_indices[:-k - 1:-1]]  # Извлекаем первые k векторов признаков
    return topk_eig_vecs

# Задание 3

In [225]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split, KFold, LeaveOneOut, StratifiedKFold, cross_val_score
from sklearn.preprocessing import scale

In [226]:
wine = datasets.load_wine()
KNC = KNeighborsClassifier()

In [227]:
kfold_max_acc = 0
kfold_max_acc_k = 0
for k in range(1, 51):
    KNC.n_neighbors = k
    curr_score = cross_val_score(KNC, wine.data, wine.target, cv=KFold(shuffle=True, random_state=42), scoring='accuracy').mean()
#     print(curr_score)
    if kfold_max_acc < curr_score:
        kfold_max_acc = curr_score
        kfold_max_acc_k = k

In [228]:
LOO_max_acc = 0
LOO_max_acc_k = 0
for k in range(1, 51):
    KNC.n_neighbors = k
    curr_score = cross_val_score(KNC, wine.data, wine.target, cv=LeaveOneOut(), scoring='accuracy').mean()
#     print(curr_score)
    if LOO_max_acc < curr_score:
        LOO_max_acc = curr_score
        LOO_max_acc_k = k


In [229]:
skfold_max_acc = 0
skfold_max_acc_k = 0
for k in range(1, 51):
    KNC.n_neighbors = k
    curr_score = cross_val_score(KNC, wine.data, wine.target, cv=StratifiedKFold(shuffle=True, random_state=42), scoring='accuracy').mean()
#     print(curr_score)
    if skfold_max_acc < curr_score:
        skfold_max_acc = curr_score
        skfold_max_acc_k = k


In [230]:
print(f'Максимальная точность классификации при использовании Kfold: {kfold_max_acc} при k = {kfold_max_acc_k}')
print(f'Максимальная точность классификации при использовании LOO: {LOO_max_acc} при k = {LOO_max_acc_k}')
print(f'Максимальная точность классификации при использовании SKfold: {skfold_max_acc} при k = {skfold_max_acc_k}')

Максимальная точность классификации при использовании Kfold: 0.7304761904761905 при k = 1
Максимальная точность классификации при использовании LOO: 0.7696629213483146 при k = 1
Максимальная точность классификации при использовании SKfold: 0.7185714285714285 при k = 1


Таким образом, лучшие результаты показал метод ближайших соседей при числе соседей равном одному и использованием LeaveOneOut кросс-валидации. При этом при всех способах валидации лучший результат получается при одном соседе, то есть элементу просто присваивается тот класс, который имеет ближайший к нему сосед с известным классом.

Проведем масштабирование признаков (wine.data -> sklearn.preprocessing.scale(wine.data)) и проделаем то же самое.

In [231]:
kfold_max_acc = 0
kfold_max_acc_k = 0
for k in range(1, 51):
    KNC.n_neighbors = k
    curr_score = cross_val_score(KNC, scale(wine.data), wine.target, cv=KFold(shuffle=True, random_state=42), scoring='accuracy').mean()
    if kfold_max_acc < curr_score:
        kfold_max_acc = curr_score
        kfold_max_acc_k = k
        
LOO_max_acc = 0
LOO_max_acc_k = 0
for k in range(1, 51):
    KNC.n_neighbors = k
    curr_score = cross_val_score(KNC, scale(wine.data), wine.target, cv=LeaveOneOut(), scoring='accuracy').mean()
    if LOO_max_acc < curr_score:
        LOO_max_acc = curr_score
        LOO_max_acc_k = k

skfold_max_acc = 0
skfold_max_acc_k = 0
for k in range(1, 51):
    KNC.n_neighbors = k
    curr_score = cross_val_score(KNC, scale(wine.data), wine.target, cv=StratifiedKFold(shuffle=True, random_state=42), scoring='accuracy').mean()
    if skfold_max_acc < curr_score:
        skfold_max_acc = curr_score
        skfold_max_acc_k = k

print('После масштабирования признаков')
print(f'Максимальная точность классификации при использовании Kfold: {kfold_max_acc} при k = {kfold_max_acc_k}')
print(f'Максимальная точность классификации при использовании LOO: {LOO_max_acc} при k = {LOO_max_acc_k}')
print(f'Максимальная точность классификации при использовании SKfold: {skfold_max_acc} при k = {skfold_max_acc_k}')

После масштабирования признаков
Максимальная точность классификации при использовании Kfold: 0.9776190476190475 при k = 29
Максимальная точность классификации при использовании LOO: 0.9831460674157303 при k = 36
Максимальная точность классификации при использовании SKfold: 0.9776190476190475 при k = 13


Видим значительное увеличение точности работы всех трех методов, потому что после нормализации все признаки имеют один порядок и большие значения не так влияют на расстояния. Вообще, для использования метода k ближайших соседей лучше всегда применять нормализацию. Все методы дали очень хороший близкий к единице результат, однако LeaveOneOut кросс-валидация осталась лучшей из трех. Также стоит отметить изменение оптимального числа соседей (смотри вывод ячейки выше).